In [1]:
import pandas as pd
import math
import requests
import numpy as np
from datetime import datetime, timedelta
from scipy.stats import genextreme
from random import choices

In [19]:
def get_token(path_to_config = '../tradier_exploration/'):
    config = pd.read_csv(f'{path_to_config}sandbox.config')
    token = config['prod_token'].iloc[0]
    
    return(token)

def get_hist_data(symbol, 
                  token,
                  start = '2000-01-01', 
                  end = datetime.today().date().isoformat(),
                  endpoint = 'https://api.tradier.com',
                  path = '/v1/markets/history'):

    response = requests.get(f'{endpoint}{path}',
        params = {'symbol':f'{symbol}', 
                  'interval': 'daily', 
                  'start': '2000-01-01', 
                  'end': f'{datetime.today().date().isoformat()}'},
        headers = {'Authorization': f'Bearer {token}', 
                   'Accept': 'application/json'})
    json_response = response.json()
    
    hist_price = pd.json_normalize(json_response['history']['day'])
    hist_price = hist_price.rename(({'date': 'Date',
                                     'open': 'Open',
                                     'close': 'Close'}),
                                    axis = 'columns')
    hist_price['Open'] = hist_price['Open'].astype('float64')
    hist_price['Close'] = hist_price['Close'].astype('float64')
    hist_price['perc_change'] = ((hist_price['Close'] - hist_price['Open']) / 
                                    hist_price['Open'])
                                    
    return(hist_price)


def get_current_price(symbol, 
                      token,
                      endpoint = 'https://api.tradier.com',
                      path = '/v1/markets/quotes'):

    response = requests.get(f'{endpoint}{path}',
        params = {'symbols': f'{symbol}', 
                  'greeks': 'false'},
        headers = {'Authorization': f'Bearer {token}', 
                   'Accept': 'application/json'})
    
    json_response = response.json()
    price = json_response['quotes']['quote']['last']
    
    return(price)

def get_simulation(symbol, 
                   token,
                   finish_date,
                   num_samples = 10000,
                   sample_size = 20000, 
                   upper_scale = 0.60, 
                   upper_shape = -0.09, 
                   lower_scale = 0.65, 
                   lower_shape = -0.1,
                   today = datetime.today().date()):

    date = {"Date": pd.date_range(today, finish_date)}

    dates = pd.DataFrame(data = date)
    dates['wday'] = dates['Date'].dt.dayofweek
    dates = dates.loc[dates['wday'] != 5]
    dates = dates.loc[dates['wday'] != 6]

    num_trading_days = dates.shape[0]
    
    start = today - timedelta(days = num_trading_days)
    start = start.isoformat()
    
    hist_price = get_hist_data(symbol, token)
    
    perc_pos = (
        sum(hist_price.loc[hist_price['Date'] > start, 'perc_change'] > 0) / 
        len(hist_price.loc[hist_price['Date'] > start, 'perc_change'])
    )

    perc_neg = 1 - perc_pos

    avg = np.mean(hist_price['perc_change'])
    std = np.std(hist_price['perc_change'])

    samp_upper = genextreme.rvs(upper_shape, 
                                loc = avg, 
                                scale = upper_scale * std,
                                size = round(sample_size * perc_pos))

    samp_lower = -1 * genextreme.rvs(lower_shape, 
                                     loc = avg, 
                                     scale = lower_scale * std,
                                     size = round(sample_size * perc_neg))

    samp = np.append(samp_upper, samp_lower)
    del samp_upper, samp_lower

    sample_col = pd.Series(range(1, num_trading_days + 1)).repeat(num_samples)
    dates_col = dates['Date'].repeat(num_samples)
    sample_values = np.array(choices(samp + 1, 
                                     k = num_samples * num_trading_days
                                     )).reshape(num_trading_days, 
                                                num_samples)

    start_price = get_current_price(symbol, token)

    sample_values[0] = start_price

    price_paths = sample_values.cumprod(axis = 0)

    final_prices = price_paths[num_trading_days - 1]
    del price_paths
    
    prob_below = sum(final_prices < start_price) / num_samples
    quantiles = np.quantile(final_prices, (0.0, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 1.0))
    
    return(prob_below, quantiles, start_price)

def get_stock_table(url = 'https://iocalc.com/api:gPr_YnZX/getstocks'):
    response = requests.get(url)

    stock_table = pd.json_normalize(response.json()['stocks'])

    return(stock_table)


In [20]:
token = get_token()
stock_table = get_stock_table()

end_date = (datetime.today().date() + timedelta(days = 30)).isoformat()
prob_change = pd.DataFrame()

for symbol in stock_table['symbol']:
    prob_below_current, quantile, current_price = get_simulation(symbol, token, end_date)
    
    tmp = pd.DataFrame({'symbol': symbol,
                        'sim_end_date': end_date,
                        'current_price': current_price,
                        'prob_below_current': prob_below_current,
                        'min_sim_price': quantile[0],
                        'percentile_5': quantile[1],
                        'percentile_10': quantile[2],
                        'percentile_25': quantile[3],
                        'median_sim_price': quantile[4],
                        'percentile_75': quantile[5],
                        'percentile_90': quantile[6],
                        'percentile_95': quantile[7],
                        'max_sim_price': quantile[8]},
                      index = [0])
    
    prob_change = pd.concat([prob_change, tmp], axis = 0, ignore_index = True)

In [21]:
prob_change

,symbol,sim_end_date,current_price,prob_below_current,min_sim_price,percentile_5,percentile_10,percentile_25,median_sim_price,percentile_75,percentile_90,percentile_95,max_sim_price
0,QYLD,2023-01-04,16.5200,0.5708,13.099398,15.394808,15.635638,16.006575,16.414290,16.803751,17.178093,17.414951,18.929200
1,CASH,2023-01-04,42.6500,0.7078,21.368608,31.973522,33.751803,36.680672,39.902098,43.320227,46.740340,48.811491,64.272967
2,VZ,2023-01-04,37.0700,0.4780,28.922905,33.522709,34.307339,35.687383,37.179282,38.718699,40.152374,41.071869,47.417749
3,DAL,2023-01-04,35.7900,0.6004,17.895103,27.448851,28.953479,31.579757,34.632523,37.741587,40.775758,42.864006,62.033491
4,HUM,2023-01-04,547.4200,0.4905,337.019162,461.220343,478.911979,512.114480,548.885334,588.562979,626.449408,651.056748,846.092917
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,OII,2023-01-04,14.9500,0.2952,7.102605,12.936054,13.573479,14.695124,16.011975,17.461562,18.902453,19.838386,31.519451
66,SBIO,2023-01-04,30.6001,0.8193,16.618783,23.774476,24.822655,26.427949,28.246444,29.984748,31.603279,32.650280,41.574610
67,RKLB,2023-01-04,4.1500,0.7886,1.318259,2.585093,2.782607,3.153604,3.596536,4.064302,4.514132,4.790573,7.005420
68,KOS,2023-01-04,6.0200,0.4048,2.936058,4.709687,5.059667,5.615288,6.262642,6.972665,7.711132,8.194180,12.647781


In [13]:
stock_table.iloc[63]

id                                71
created_at             1643141758315
name               Pacer Funds Trust
symbol                          SRVR
price                          37.29
target                           0.0
marketcap                 1401745500
week52high                      43.5
week52low                      31.89
dividendyield               0.013691
day200movingavg                40.54
day50movingavg                 40.61
exdividend                2021-12-23
nextearnings                    None
eps                                 
pe                                  
beta                                
change_1yr         0.050333739137736
change_ytd         -0.13285582255083
pricetargethigh                  0.0
pricetargetlow                   0.0
symboltype                          
analysts                           0
currency                            
totaltracking                      0
Name: 63, dtype: object

In [18]:
get_hist_data('SRVR', token)

TypeError: unsupported operand type(s) for -: 'float' and 'str'